In [1]:
from stackapi import StackAPI

SITE = StackAPI('stackoverflow')

qids = [
    11881165, 11941492, 13647222, 18172851, 49583055, 49592930, 49572546, 13261175, 
    13793321, 14085517, 11418192, 49567723, 13261691, 13659881, 13807758, 34365578, 
    10982266, 11811392, 49581206, 12065885, 13576164, 14023037, 53762029, 21982987, 
    39656670, 23321300
]


In [2]:
autopandas_questions = SITE.fetch('questions/{ids}', ids=qids, filter='withbody')

In [3]:
print(autopandas_questions.keys())

dict_keys(['backoff', 'has_more', 'page', 'quota_max', 'quota_remaining', 'total', 'items'])


In [4]:
ap_questions = autopandas_questions['items']
print(ap_questions[0].keys())

dict_keys(['tags', 'owner', 'is_answered', 'view_count', 'accepted_answer_id', 'answer_count', 'score', 'last_activity_date', 'creation_date', 'last_edit_date', 'question_id', 'content_license', 'link', 'title', 'body'])


In [5]:
accepted_answer_ids = [a['accepted_answer_id'] for a in ap_questions if 'accepted_answer_id' in a.keys()]
print(len(accepted_answer_ids))
answers = SITE.fetch('answers/{ids}', ids=accepted_answer_ids, filter='withbody')["items"]
print(answers[0].keys())

24
dict_keys(['owner', 'is_accepted', 'score', 'last_activity_date', 'last_edit_date', 'creation_date', 'answer_id', 'question_id', 'content_license', 'body'])


In [12]:
from bs4 import BeautifulSoup as bs
import re
import nltk

def min_word_filter(texts, min_word=3):
    filtered_texts = []
    for t in texts:
        if len(nltk.wordpunct_tokenize(t)) >= min_word:
            filtered_texts.append(t)
            pass
        pass
    return filtered_texts

def extract_code(text, filters):
    soup = bs(text)
    all_code = [code.text for code in soup.find_all('code')]
    for f in filters:
        all_code = f(all_code)
    return all_code
    pass

def add_answer_to_code(questions, answer):
    for idx, q in enumerate(questions):
        if q["question_id"] == answer["question_id"]:
            codes =  extract_code(answer["body"], filters=[min_word_filter])
            questions[idx]["code"] = codes
            break
            pass
        pass
    pass

for a in answers:
    add_answer_to_code(ap_questions, a)
    pass

for q in ap_questions:
    if "code" not in q.keys():
        q["code"] = []
            

In [13]:
def extract_description(html):
    soup = bs(html)
    [code.extract() for code in soup.find_all('code')]
    text = re.sub("[ \n\t]+", " ", soup.text)
    return text
    pass


In [14]:
test_examples = []

for example_id, q in enumerate(ap_questions):
    ques_id = q["question_id"]
    qtitle = q["title"]
    qdesc = extract_description(q["body"])
    test_examples.append(
        {
            'id': ques_id,
            'query': qtitle.strip().lower() + " " + qdesc.strip().lower(),
            "apis": [],
            'link': q['link'],
            "example": q["code"]
        }
    )

In [15]:
import json 

fp = open("dpr_exp/autopandas_test.json", "w")
json.dump(test_examples, fp, indent=4)
fp.close()

In [16]:
print(json.dumps(test_examples, indent=4))

[
    {
        "id": 18172851,
        "query": "deleting dataframe row in pandas based on column value i have the following dataframe: i need to remove the rows where is equal to . what's the most efficient way to do this?",
        "apis": [],
        "link": "https://stackoverflow.com/questions/18172851/deleting-dataframe-row-in-pandas-based-on-column-value",
        "example": [
            "df = df[df.line_race != 0]\n"
        ]
    },
    {
        "id": 21982987,
        "query": "mean per group in a data.frame i have a and i need to calculate the mean per group (i.e. per , below). my desired output is like below, where the values for and are the group means. please disregard the value, i have made it up for the example.",
        "apis": [],
        "link": "https://stackoverflow.com/questions/21982987/mean-per-group-in-a-data-frame",
        "example": [
            "d <- read.table(text=\n'Name     Month  Rate1     Rate2\nAira       1      12        23\nAira       2      18